# History of the US 

## Using GitHub data

First we list the available sources on github.

In [ ]:
val geoJsonsDir = "https://api.github.com/repos/poezn/us-history-maps/contents/GeoJSON?ref=master"
val listingString = scala.io.Source.fromURL(geoJsonsDir).getLines.mkString
val listing = play.api.libs.json.Json.parse(listingString)
val play.api.libs.json.JsArray(arr) = listing
()

Here is the list of recorded changes in the US states ownership

In [ ]:
val geoJsons = arr.map(x => (x \ "name").as[String])
                  .filterNot(_ == ".DS_Store")
                  .filterNot(_ startsWith "mapgroup")
                  .filter(_.nonEmpty)
                  .map(x => (x.takeWhile(_.isDigit).toInt, s"https://raw.githubusercontent.com/poezn/us-history-maps/master/GeoJSON/$x"))
                  .sortBy(_._1)
()

In [ ]:
geoJsons.map(_._1).max

## Quick view: distribution of changes

This should give us some help to understand how the data is structured

In [ ]:
val minDate = geoJsons.sorted.head._1
LineChart(geoJsons.map(_._1) zip geoJsons.map(_._1 - minDate).toList.sorted)

Here is a chart showing how often the states changed per year.

In [ ]:
BarChart(geoJsons.map(_._1)
                 .groupBy(_.toString.take(6))
                 .toList
                 .map(x => (x._1, x._2.size))
                 .sortBy(_._2)
        )

## American Civil War

In [ ]:
val time = geoJsons .map(_._1)
                    .map{ x => 
                      "" + x.toString.take(6)+
                        (1+(x%10)*5).toString.reverse.padTo(2, "0").reverse.mkString
                    }
()

In [ ]:
val `American Civil War` = time .dropWhile(!_.toString.startsWith("1861"))
                                .takeWhile(!_.toString.startsWith("1866"))
CustomC3Chart(time zip geoJsons.map(_._1),
             chartOptions = s"""
             { 
               data: { 
                 x: '_1',
                 xFormat: '%Y%m%d',
                 type: 'scatter'
               },
               axis: {
                 x: { 
                   type: 'timeseries',
                   tick: { 
                     format: '%Y-%b-%W' , 
                     rotate: 45
                   } 
                 }
               },
               regions: [
                 {
                   axis: 'x', 
                   start: "${`American Civil War`.head}", 
                   end: "${`American Civil War`.last}", 
                   class: 'regionX'
                 },
               ]
             }
             """)


# Wouldn't be great to see this live?

Collect the data from GeoJSON.

In [ ]:
val head = geoJsons.head
def extractData(u:String) = {
  val h = scala.io.Source.fromURL(u).getLines.mkString
  val fs =  GeoChart.parseGeoJSON(h) match {
              case fc:org.wololo.geojson.FeatureCollection => fc.getFeatures
            }
  fs.map { f =>
    (f, Option(f.getProperties.get("COUNTRY")).map(_.toString).getOrElse("Disputed"))
  }
  
}
val h = extractData(head._2)
()

In [ ]:
var countries = h.map(_._2).distinct.toList
val colors = List(
  "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c", "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5", 
  "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f", "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5"
)
def mapping = (countries zip colors).toMap
()

In [ ]:
def colorizeCountries(cs:List[String]):List[String] = mapping .filter(xs => cs contains (xs._1))
                                                              .map{ 
                                                                case (n, c) => s"""<span style="color: $c">$n</span>"""
                                                              }.toList
val u:DataConnectedWidget[String] = ul(20, colorizeCountries(countries))
def updateLegend(cs:List[String]) = u(colorizeCountries(cs))
()

In [ ]:
def toGeoData(d:Array[(org.wololo.geojson.Feature, String)]) = d.map(x => (x._1, mapping(x._2)))
val g = GeoChart(toGeoData(h), geometryField=Some("_1"), fillColorField=Some("_2"), sizes=(800,800))
()

In [ ]:
val d:SingleConnectedWidget[String] = out
val dt = new java.text.SimpleDateFormat("MMM YYYY")
val c = java.util.Calendar.getInstance
c.set(java.util.Calendar.DATE, 1)
def toDate(v:Int) = {
  val i = v.toString.take(6)
  c.set(java.util.Calendar.YEAR, i.take(4).toInt)
  c.set(java.util.Calendar.MONTH, i.drop(4).take(2).toInt)
  dt.format(c.getTime)
}
d("In " + toDate(head._1))
()

In [ ]:
val b = BarChart(h.groupBy(_._2).mapValues(_.size).toList, sizes=(200, 200))
def updateBars(xs:Array[String]) = b.applyOn(xs.groupBy(x=>x).map{ x => (x._1, x._2.size)}.toList)

## Dashboard

In [ ]:
containerFluid(List(
  List((d, 2), (u, 5), (b, 5)),
  List((html(<div><br/><hr/><br/></div>), 12)),
  List((g.asInstanceOf[Widget], 12))
))

In [ ]:
import scala.concurrent.Future
import scala.concurrent.ExecutionContext.Implicits.global
var msBetweenExp = 500
Future {
  geoJsons.tail.foreach{ x =>
    val dt = extractData(x._2)
    val cs = dt.map(_._2).distinct.toList
    countries = countries ::: (cs diff countries) 
    updateLegend(cs)
    updateBars(dt.map(_._2))
    val j = toGeoData(dt)
    d("In " + toDate(x._1))
    g.applyOn(j)
    Thread.sleep(msBetweenExp)
  }
}

In [ ]:
msBetweenExp = 10